In [1]:
import pandas as pd 
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
import pickle
from copy import deepcopy
from pyentrp import entropy as ent
import warnings
import datetime
warnings.filterwarnings("ignore")
import pandas_ta as ta
import re 

from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import  accuracy_score, classification_report, roc_auc_score, recall_score, precision_score,f1_score 

In [2]:

industrydic = pd.read_csv('us_stock_meta.csv')
industrydic = dict(zip(industrydic.Ticker,industrydic.industry))

In [3]:
# df =   pd.read_pickle('us_equity_data_final_spx.pkl')
# df = df.loc['2018-01-01':]
# with open('production_data_test.pkl','wb') as pkl :
#     pickle.dump(df, pkl)

In [4]:
# df = pd.read_pickle('us_equity_data_final_spx.pkl')
# newdf = pd.read_pickle('us_equity_data_updated.pickle')
# df = df.append(newdf)

In [13]:
df =   pd.read_pickle('production_data_test.pkl')

In [18]:
df = pd.read_csv('production_data_test.csv')

In [19]:
df

,Unnamed: 0,A UN Equity,A UN Equity.1,A UN Equity.2,A UN Equity.3,A UN Equity.4,A UN Equity.5,A UN Equity.6,A UN Equity.7,A UN Equity.8,...,ZTS UN Equity.15,ZTS UN Equity.16,ZTS UN Equity.17,ZTS UN Equity.18,ZTS UN Equity.19,ZTS UN Equity.20,ZTS UN Equity.21,ZTS UN Equity.22,ZTS UN Equity.23,ZTS UN Equity.24
0,NaN,ASSET_TURNOVER,CASH_RATIO,EQY_SH_OUT,HIST_CALL_IMP_VOL,HIST_PUT_IMP_VOL,IS_EPS,MARKET_CAPITALIZATION_TO_BV,OPEN_INT_TOTAL_CALL,OPEN_INT_TOTAL_PUT,...,PX_LOW,PX_TO_BOOK_RATIO,PX_VOLUME,RETURN_COM_EQY,RETURN_COM_EQY_1,RETURN_ON_ASSET,RETURN_ON_INV_CAPITAL,SHORT_AND_LONG_TERM_DEBT,TOT_DEBT_TO_TOT_ASSET,TURNOVER
1,2018-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-01-02,NaN,NaN,323.018,21.408,21.408,NaN,4.52,9655.0,29580.0,...,71.47,19.7116,606012.0,NaN,NaN,NaN,NaN,NaN,NaN,43529900.0
3,2018-01-03,NaN,NaN,323.018,22.263,22.263,NaN,4.635,11946.0,49084.0,...,71.4,19.8023,615467.0,NaN,NaN,NaN,NaN,NaN,NaN,44315620.0
4,2018-01-04,NaN,NaN,323.018,22.543000000000003,22.543000000000003,NaN,4.6002,13124.0,49437.0,...,72.26,19.9204,806599.0,NaN,NaN,NaN,NaN,NaN,NaN,58528120.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1340,2021-04-08 00:00:00,NaN,NaN,304.698,24.727,24.727,NaN,8.2891,8700,9755,...,158.64,20.178,574746,NaN,NaN,NaN,NaN,NaN,NaN,9.19226e+07
1341,2021-04-09 00:00:00,NaN,NaN,304.698,24.722,24.722,NaN,8.3665,8687,9835,...,158.47,20.1591,947109,NaN,NaN,NaN,NaN,NaN,NaN,1.51307e+08
1342,2021-04-12 00:00:00,NaN,NaN,304.698,25.114,25.114,NaN,8.3259,8694,9901,...,158.56,20.4945,751534,NaN,NaN,NaN,NaN,NaN,NaN,1.21912e+08
1343,2021-04-13 00:00:00,NaN,NaN,304.698,24.782,24.782,NaN,8.4337,8960,9941,...,161.68,20.6156,887748,NaN,NaN,NaN,NaN,NaN,NaN,1.45053e+08


In [5]:
# with open('us_equity_data_final_spx.pkl', 'wb') as pkl:
#     pickle.dump(df, pkl)

In [4]:
class read_data(): 
    def __init__(self ): 
        self.df =   pd.read_pickle('production_data_test.pkl')
        #self.df =   pd.read_pickle('dummydf.pickle')
        self.df.sort_index(axis = 1, inplace = True)
        self.df.fillna(method ='ffill', inplace = True)
        self.df = self.df.apply(pd.to_numeric)
        self.df.index = pd.to_datetime(self.df.index)
        
        self.meta = pd.read_csv('us_stock_meta.csv')
        
        self.daily_rts = self.df.loc[:,self.df.columns.get_level_values(1).isin(['PX_LAST'])]
        self.daily_rts.columns = self.daily_rts.columns.droplevel(1)
        
        self.daily_rts = self.daily_rts/self.daily_rts.shift(1)  -1

In [5]:
class feature_engineering(): 
    def __init__(self) : 
         read_data.__init__(self)
            
    def calculate_y_label(self, time_horizon,  threshold,join_original): 
        """This function generates y label for classification model based on the time horizon and return threshold, 
        if the return for the next x days exceeds y percent, label it as 1
        df: dataframe
        time_horizon: int, time horizon for the window 
        threshold: return threshold to see if exceed
        join_original: join with original dataframe
        df = calculate_y_label(df, 15, 0.05, True) """
        price = self.df.loc[:, self.df.columns.get_level_values(1).isin(['PX_LAST'])]
        rst = {} #pd.DataFrame(index = price.index, columns = price.columns )
        for i in range(0, len(price)-time_horizon+1):
            window = price.iloc[i: i+time_horizon]
            window = window/window.iloc[0]-1 
            window[window >= threshold] =1
            window[window<= (-1) * threshold] = -1
            window[(window<threshold) & (window> (-1) *threshold)] =0 
            window[window == np.nan] = np.nan
            date = price.index[i]
            rst[date] = window.sum()
        #     data ['y_direction'].loc[date] = window.sum()
        rst_df = pd.concat(rst, axis = 1).transpose()

        i = rst_df.columns.levels[1]  
        j = ['y' +'_' + str(time_horizon)  + '_d' for x in i ]
        rename_dict = dict(zip(i, j))
        rst_df.rename(columns=rename_dict, level=1,inplace = True)

        if join_original == True: 
            return  self.df.join(rst_df)
        else:
            return rst_df
        return rstdf
    
    def calculate_structural_break (self, flds, join_original):
        """This function calculates x day changes for multiindex dataframe from pdblp
        df:dataframe, input dataframe
        flds:list, fields that you want to performe calculation on --["PX_LAST", "FUND_FLOW",'VOLUME']
        days:list,  x days of changes ,[5,15]
        join_original: True or False, return calculated dataframe joined to the original df
        eg: calculate_structural_break(df, ['PX_LAST','VOLUME','FUND_FLOW','PX_HIGH'], True)"""
        #t,t1,tn = 252,251,242
        t,t1,tn = 30,60,90
        y = np.log( self.df.loc[:,self.df.columns.get_level_values(1).isin(flds)])
        sigmasq = (( (y - y.shift(1))**2).rolling(t1).sum() )* (t-1)**(-1) 
        rst_df = (y-y.shift(3*26)) * (np.sqrt(sigmasq) * np.sqrt(tn)) **(-1)

        i = rst_df.columns.levels[1]   # OP's suggestion, for more flexibility!
        j = [x + '_sb' for x in i ]
        rename_dict = dict(zip(i, j))
        rst_df.rename(columns=rename_dict, level=1,inplace = True)

        if join_original == True: 
            return self.df.join(rst_df)
        else:
            return rst_df   

    def calculate_group_diff(self, flds ,tkr_group,join_original ): 
        """This function calculates mean row difference by each group,
        the purpose of this is to compare data against group performance
        df: dataframe
        flds: list, fields to perform calculation on
        tkr_group:list,  the group to perform mean row difference,
        join_original:boolean, true or false, join with original dataframe

        calculate_group_diff(df, ['PX_LAST_5_dchg','VOLUME_5_dchg','FUND_FLOW_5_d_chg'],
                         list(meta[meta['industry'] == 'Thematic']['Ticker'].unique()), True  )"""
        rst = defaultdict()

        for i_flds in flds: 
            subdf = self.df[tkr_group].loc[:, self.df[tkr_group].columns.get_level_values(1).isin([i_flds])]
            rst[i_flds] = subdf.sub(subdf.median(axis = 1), axis=0)
            i = subdf.columns.levels[1]   # OP's suggestion, for more flexibility!
            j = [x +  '_ab' for x in i ]
            rename_dict = dict(zip(i, j))
            rst[i_flds].rename(columns=rename_dict, level=1,inplace = True)

        rst_df = pd.concat(rst.values(),axis=1)
        if join_original == True: 
            return self.df.join(rst_df)
        else:
            return rst_df
        
    def calculate_group_diff_agg(self,flds, agg_type,join_original ):
        """ This function calculates aggregated difference by group with the calculate_group_diff function
        df: dataframe, comparing all companies against group benchmark(mean)
        flds: list, fields to perform calculation on 
        agg_type: string, which field to perform aggregation
        join_original: boolean, True or False to join the original dataframe
        calculate_group_diff_agg(df,['PX_LAST'], 'industry',True)"""
        rst = defaultdict()
        for agg_group in set([x[0] for x in self.df.columns]): 
            tkr_group = list(self.meta[self.meta[agg_type] == agg_group]['Ticker'].unique())
            rst[agg_group]  = self.calculate_group_diff( flds,
                            tkr_group, False  )
        rst_df = pd.concat(rst.values(),axis=1)
        if join_original == True: 
            return self.df.join(rst_df)
        else:
            return rst_df
        
    # calculate the difference between open interest call and put
    def calculate_col_diff (self,col1, col2, col3): 
        """ this function creates difference between two columns in pandas multiindex dataframe-> col3 = col1 - col2
        self.df: dataframe
        col1, col2, col3: str, column name
        calculate_col_diff (self.df,'OPEN_INT_TOTAL_CALL', 'OPEN_INT_TOTAL_PUT', 'OPEN_INT_DIFF')
        """
        for i in set([x[0] for x in self.df.columns]) : 
            try: 
                self.df[i, col3] = self.df[i, col1] - self.df[i, col2 ]
            except: 
                self.df[i, col3 ]  = np.nan
        return self.df

    def calculate_xd_chg(self, flds, days, join_original):
        """This function calculates x day changes for multiindex dataframe from pdblp
        df:dataframe, input dataframe
        flds:list, fields that you want to performe calculation on --["PX_LAST", "FUND_FLOW",'VOLUME']
        days:list,  x days of changes ,[5,15]
        join_original: True or False, return calculated dataframe joined to the original df
        eg: calculate_xd_chg(df, ['PX_LAST','VOLUME','FUND_FLOW','PX_HIGH'], [5,15,260],True)"""
        rst = defaultdict()

        for k_days in days:
            subdf = self.df.loc[:,self.df.columns.get_level_values(1).isin(flds)]
            rst[k_days] = subdf/subdf.shift(k_days)-1

            i = subdf.columns.levels[1]   # OP's suggestion, for more flexibility!
            j = [x + '_' + str(k_days) + '_dchg' for x in i ]
            rename_dict = dict(zip(i, j))
            rst[k_days].rename(columns=rename_dict, level=1,inplace = True)
        rst_df = pd.concat(rst.values(),axis=1)
        if join_original == True: 
            return self.df.join(rst_df)
        else:
            return rst_df
    
    def calculate_bollinger_band(self, flds, join_original):
        """This function calculates x day moving average for multiindex dataframe from pdblp
        df:dataframe, input dataframe
        flds:list, fields that you want to performe calculation on --["PX_LAST", "FUND_FLOW",'VOLUME']
        days:list,  x days of changes ,[5,15]
        join_original: True or False, return calculated dataframe joined to the original df
        eg: calculate_xd_ma(df, ['PX_LAST','VOLUME','FUND_FLOW','PX_HIGH'], [5,15,260],True)"""
        rstdic =defaultdict( ) 

        subdf = self.df.loc[:,self.df.columns.get_level_values(1).isin(flds)]
        rstdic['upper']=( subdf.rolling(20).mean() )  + 2 *( subdf.rolling(20).std())
        rstdic['lower']=( subdf.rolling(20).mean() )  - 2 *( subdf.rolling(20).std())
        for k in rstdic.keys(): 
            i = subdf.columns.levels[1]   # OP's suggestion, for more flexibility!
            j = [x + '_bollinger_' + k for x in i ]
            rename_dict = dict(zip(i, j))
            rstdic[k].rename(columns=rename_dict, level=1,inplace = True)
        rstdf = pd.concat(rstdic.values(), axis = 1)
        if join_original == True: 
            return self.df.join(rstdf)
        else:
            return rstdf
        
    def calculate_xd_ma(self, flds, days, join_original):
        """This function calculates x day moving average for multiindex dataframe from pdblp
        df:dataframe, input dataframe
        flds:list, fields that you want to performe calculation on --["PX_LAST", "FUND_FLOW",'VOLUME']
        days:list,  x days of changes ,[5,15]
        join_original: True or False, return calculated dataframe joined to the original df
        eg: calculate_xd_ma(df, ['PX_LAST','VOLUME','FUND_FLOW','PX_HIGH'], [5,15,260],True)"""
        rst = defaultdict()

        for k_days in days:
            subdf = self.df.loc[:,self.df.columns.get_level_values(1).isin(flds)]
            rst[k_days] = subdf.rolling(k_days).mean()

            i = subdf.columns.levels[1]   # OP's suggestion, for more flexibility!
            j = [x + '_' + str(k_days) + '_dma' for x in i ]
            rename_dict = dict(zip(i, j))
            rst[k_days].rename(columns=rename_dict, level=1,inplace = True)
        rst_df = pd.concat(rst.values(),axis=1)
        if join_original == True: 
            return self.df.join(rst_df)
        else:
            return rst_df
        
    def calculate_xd_vol(self, flds, days, join_original):
        """This function calculates x day volatility for multiindex dataframe from pdblp
        df:dataframe, input dataframe
        flds:list, fields that you want to performe calculation on --["PX_LAST", "FUND_FLOW"]
        days:list,  x days of changes ,[5,15]
        join_original: True or False, return calculated dataframe joined to the original df
        eg: calculate_xd_vol(df, ['PX_LAST','FUND_FLOW'], [5,15,260],True)"""
        rst = defaultdict()

        for k_days in days:
            subdf = self.df.loc[:,self.df.columns.get_level_values(1).isin(flds)]
            rst[k_days] = subdf.rolling(k_days).std()

            i = subdf.columns.levels[1]   # OP's suggestion, for more flexibility!
            j = [x + '_' + str(k_days) + '_dstd' for x in i ]
            rename_dict = dict(zip(i, j))
            rst[k_days].rename(columns=rename_dict, level=1,inplace = True)
        rst_df = pd.concat(rst.values(),axis=1)
        if join_original == True: 
            return self.df.join(rst_df)
        else:
            return rst_df
        
    def calculate_xd_sharpe(self, flds, days, join_original):
        """This function calculates x day period sharpe for multiindex dataframe from pdblp
        df:dataframe, input dataframe
        flds:list, fields that you want to performe calculation on --["PX_LAST"]
        days:list,  x days of changes ,[5,15]
        join_original: True or False, return calculated dataframe joined to the original df
        eg: calculate_xd_sharpe(df, ['PX_LAST'], [5,15,260],True)"""
        rst = defaultdict()
        subdf = self.df.loc[:,self.df.columns.get_level_values(1).isin(flds)]
        subdf_rts = subdf/ subdf.shift(1)-1
        for k_days in days:
            rst[k_days] =( np.sqrt(260)) * (subdf_rts.rolling(k_days).mean() / subdf_rts.rolling(k_days).std() )
            i = subdf.columns.levels[1]   # OP's suggestion, for more flexibility!
            j = [x + '_' + str(k_days) + '_dsharpe' for x in i ]
            rename_dict = dict(zip(i, j))
            rst[k_days].rename(columns=rename_dict, level=1,inplace = True)
        rst_df = pd.concat(rst.values(),axis=1)
        if join_original == True: 
            return self.df.join(rst_df)
        else:
            return rst_df
        
    def calculate_time_variable(self): 
        self.df['dayofweek'] = self.df.index.dayofweek
        self.df['weekofmonth'] = list(pd.Series(self.df.index).apply(lambda d: (d.day-1) // 7 + 1))
        self.df['weekofyear'] = self.df.index.weekofyear
        return self.df

    def convert_categorical(self, categorical_cols): 
        """this function converts categorical columns into numeric columns before classification
        self.df : dataframe
        categorical_cols: list of categorical columns to convert to"""
        for i in categorical_cols : 
            self.df[i] = pd.factorize( self.df[i] )[0]
        return self.df

    def generate_final_df(self, varlist): 
        subdf = self.df.loc[:,self.df.columns.get_level_values(1).isin(varlist)]
        finaldf = pd.DataFrame()
        for i in list(set([x[0] for x in subdf.columns])) : 
            subdf[i, 'ticker'] = i
            subdf [i, 'industry'] = industrydic[i]
            finaldf = finaldf.append(subdf[i])

        self.df = finaldf.apply(lambda x: x.replace( [np.inf, -np.inf], np.nan))
        self.df = self.calculate_time_variable()
        self.df = self.convert_categorical( ['industry'])
        self.df = self.df.sort_index()
        
        return self.df


    def convert_y_label(self, label): 
        """label->list"""
        for i in label : 
            self.df[i] = np.where(self.df[i]<0, -1 ,self.df[i])
            self.df[i] = np.where(self.df[i]>0, 1 ,self.df[i])
        return self.df

In [6]:
class feature_selection() :
    def __init__(self, input_df):
        self.df = input_df
        
    def get_rf_feature_importance(self, varlist, y):
        """This function returns random forest feature importance data in a dataframe
        varlist: list, a list of features to put into random forest
        y: target variable
        df: input dataframe
        get_rf_feature_importance ( ['cap','OPEN_INT_DIFF','FUND_FLOW_5_dchg','VOLUME_5_dchg','VOLUME_260_dchg'],
        'y_15_d',finaldf)"""
        inputdf = (self.df[varlist + [y] ].dropna())
        cutoff = int( 0.8 * len(inputdf))
        xtrain,ytrain = inputdf.iloc[:cutoff][varlist], inputdf.iloc[:cutoff][[y]]
        xtest,ytest =  inputdf.iloc[cutoff:][varlist], inputdf.iloc[cutoff:][[y]]

        rf = RandomForestClassifier(n_estimators= 500,n_jobs=1 ).fit(xtrain, ytrain)
        y_pred = rf.predict(xtest)
        testdf =  inputdf.iloc[cutoff:][varlist + [y]]
        testdf['y_pred'] = y_pred
        feature_importances = pd.DataFrame(rf.feature_importances_,
                                       index = xtrain.columns,
                                        columns=['importance']).sort_values('importance',  ascending=False)
        return feature_importances

        
    def get_precision_feature_importance(self, varlist, y): 
        """This function pass in features one by one from the varlist to build random forest and compare precision score to select 
        the best feature"""

        precision_df = pd.DataFrame(index = varlist, columns= ['precision'])
        precision_df.index.name = 'field'

        for x in varlist: 
            inputdf =  (self.df[  [x,y] ].dropna())
            cutoff = int( 0.8 * len(inputdf))
            xtrain,ytrain = inputdf.iloc[:cutoff][[x]], inputdf.iloc[:cutoff][[y]]
            xtest,ytest =  inputdf.iloc[cutoff:][[x]], inputdf.iloc[cutoff:][[y]]

            rf = RandomForestClassifier(n_estimators= 500).fit(xtrain, ytrain)
            y_pred = rf.predict(xtest)
            precision_df.loc[x] = precision_score(ytest, y_pred, average = 'weighted')
        precision_df = precision_df.sort_values(by  ='precision', ascending = False, )
        return precision_df
    
    def select_features(self, varlist, y, method): 
        """This function ranks feature by importance using the get_rf_feature_importance and  get_precision_feature_importance
        function
        varlist : list 
        y: str
        df: dataframe
        method: str  """
        if method == 'feature_importance' : 
            feature_importance = self.get_rf_feature_importance ( varlist, y)
            return feature_importance
        if method == 'precision_importance':
            precision_importance = self.get_precision_feature_importance( varlist, y).sort_values(by = 'precision', ascending = False)
            return precision_importance
        if method =='both':
            feature_importance = self.get_rf_feature_importance ( varlist, y)
            precision_importance =  self.get_precision_feature_importance( varlist, y).sort_values(by = 'precision', ascending = False)
            total = precision_importance.join(feature_importance)
            total['total'] = total.sum(axis =1 )
            total = total.sort_values(by = 'total', ascending = False)
            return total



In [7]:
class model_building(): 
    def __init__(self, df): 
        self.df = df
        
    def build_models(self, feature_list, y, model_param ): 
        """This function build all classification models for model selection purpose
        it returns, model, prediction and confusion matrix
        feature_list: list, a list of features to feed into the dataframe
        y: string, for y variable
        df: input dataframe
        build_models(feature_ranking , 4, 'y_15_d', finaldf.tail(400))"""

        inputdf = self.df[feature_list + [y,'ticker']].dropna().sort_index()
        print(len(inputdf))
        print(feature_list)
        modeldic, modelpred = {},{}
        scores = pd.DataFrame()
        cutoff = int( 0.8 * len(inputdf))
        xtrain,ytrain = inputdf.iloc[:cutoff][feature_list], inputdf.iloc[:cutoff][[y]]
        xtest,ytest =  inputdf.iloc[cutoff:][feature_list], inputdf.iloc[cutoff:][[y]]

        for i , j in model_param.items():
            modeldic[i] = j.fit(xtrain, ytrain)
            testdf =  inputdf.iloc[cutoff:][  [y,'ticker']]
            testdf['y_pred'] = modeldic[i].predict(xtest)
            try:
                testdf[['p(-1)','p(0)','p(1)']] = modeldic[i].predict_proba(xtest)
            except:
                None
            modelpred[i]  = testdf
            scores.loc[i,'accuracy'] = accuracy_score (ytest,modelpred[i]['y_pred']  )
            scores.loc[i,'precision'] = precision_score (ytest,modelpred[i]['y_pred'] , average = 'weighted')
            scores.loc[i,'recall'] = recall_score (ytest,modelpred[i]['y_pred']  , average = 'weighted' )
            scores.loc[i,'f1'] = f1_score (ytest,modelpred[i]['y_pred']  , average = 'weighted')
        return modeldic , modelpred , scores

    
    ## full backtest model
    def build_models_full(self, feature_list, y, model_param ): 
        modeldic, modelpred = {},{}
        inputdf = self.df[feature_list + [y,'ticker']].dropna().sort_index()
        print(len(inputdf))
        print(feature_list)
        modeldic, modelpred = {},{}
        scores = pd.DataFrame()

        for i in model_param.keys(): 
            modelpred[i] = pd.DataFrame()

        for idx  in range(0,10,2):
            # specify start and end interval for test set
            int_start , int_end = int( (idx/10) * len(inputdf)), int( (idx/10 + 0.2) * len(inputdf))
            xtest,ytest =  inputdf.iloc[int_start:int_end][feature_list], inputdf.iloc[int_start:int_end][[y]]
            xtrain, ytrain = inputdf[~inputdf.index.isin(xtest.index)][feature_list] ,inputdf[~inputdf.index.isin(ytest.index)][[y]]

            for i , j in model_param.items():
                modeldic[i] = j.fit(xtrain, ytrain)
                testdf =  inputdf.iloc[int_start:int_end][ [y,'ticker']]
                testdf['y_pred'] = modeldic[i].predict(xtest)
                try:
                    testdf[['p(-1)','p(0)','p(1)']] = modeldic[i].predict_proba(xtest)
                except:
                    None
                modelpred[i] = modelpred[i].append( testdf ) 

        for i in model_param.keys():
            scores.loc[i,'accuracy'] = accuracy_score (inputdf[[y]],modelpred[i]['y_pred']  )
            scores.loc[i,'precision'] = precision_score (inputdf[[y]],modelpred[i]['y_pred'] , average = 'weighted')
            scores.loc[i,'recall'] = recall_score (inputdf[[y]],modelpred[i]['y_pred']  , average = 'weighted' )
            scores.loc[i,'f1'] = f1_score (inputdf[[y]],modelpred[i]['y_pred']  , average = 'weighted')
        return modeldic , modelpred , scores
    
    def build_final_model(self, feature_list, y, model_param): 
        modeldic, modelpred = {},{}
        inputdf = self.df[feature_list + [y,'ticker']].dropna().sort_index()
        X_df, y_df = inputdf[feature_list], inputdf[[y]]
        for i , j in model_param.items():
            modeldic[i] = j.fit(X_df, y_df)
        return modeldic
    
    def get_latest_prediction(self, model,df, sort_order) :  

        df =  df.loc['2021-01-01':].reset_index().set_index(['ticker','date'])
        df = df.dropna()
        df['y_pred'] =model.predict( df)
        final_pred = df[['y_pred']].reset_index()
        final_pred = final_pred[ final_pred.date==list(final_pred.date)[-1]]
        sort_order.index.name = 'ticker'
        final_pred = final_pred.merge(sort_order.reset_index(), how = 'outer').sort_values(by = 'correct', ascending = False)
        final_pred = final_pred[final_pred['y_pred']!=0].set_index('date')
        return final_pred

In [8]:
class backtest(): 
    def portfolio_stats_matrix(self, daily_rts, input_signal, strategy, weights_type):
        """inputsignal :is a weekly dataframe
        This function generates backtest result and stats, give asset daily returns, input signal/weights
        daily_rts: dataframe of daily returns of assets 
        input_signal: dataframe of asset weights/sigmal
        strategy: str -- 'long only' to be expanded to 'long short'
        weights_type: str, equal or weighted, to decide if the signal weights"""
        rf = 0.02
        input_weights = input_signal.reset_index().set_index(['date','ticker'])[['y_pred']].unstack()
        input_weights.columns  = input_weights.columns.droplevel(0)
        input_weights = input_weights.resample('D').first().fillna(method = 'ffill')
        if strategy == 'long only':
            input_weights[input_weights<0] = 0
        if weights_type =='weighted': 
            input_weights = input_weights.div(input_weights.sum(axis=1), axis=0)
        elif weights_type =='equal': 
            input_weights[input_weights>0]=1
            input_weights= input_weights.div(input_weights.sum(axis=1), axis=0)

        #input_weights = input_weights [input_weights!= 0].dropna(how='all',axis=1).fillna(0).sort_index()

        returns = daily_rts[input_weights.columns].loc[input_weights.index[0]: input_weights.index[-1]]
        returns['portfolio_returns'] = (returns*input_weights).sum(axis = 1)
        returns['portfolio_cum_returns'] = (returns['portfolio_returns']+1).cumprod() -1

        port_mean_returns = returns['portfolio_returns'].mean()*260
        port_total_returns =  returns['portfolio_cum_returns'][-1]
        port_sigma = returns['portfolio_returns'].std() * np.sqrt(260)
        port_sharpe = (port_mean_returns - rf)/ port_sigma
        return round(port_mean_returns,2), round(port_sigma,2),round(port_sharpe,2),round(port_total_returns,2), returns[['portfolio_cum_returns']]


    def calculate_model_score(self, modelpred):
        """This function calculates the correction score for each model in the model pred dictioanry"""
        rst = pd.DataFrame()
        for i in modelpred.keys(): 
            label = [x for x in modelpred[i].keys() if x !='ticker' and x!= 'y_pred'][0]
            subdf = modelpred[i][modelpred[i][label]!=0]
            rst.loc['correct',i] = len(subdf[subdf[label] == subdf['y_pred']]) / len(subdf)
            rst.loc['wrong',i] = len(subdf[(subdf[label] != subdf['y_pred']) & (subdf['y_pred']!=0)])/ len(subdf)
            rst.loc['miss',i] =  (len(subdf[(subdf[label] != subdf['y_pred']) & (subdf['y_pred']==0)])/ len(subdf))
        return rst

    def calculate_fund_score(self, modelpred): 
        rst = {}
        label = [x for x in modelpred['gb'].keys() if '_d' in x][0]
        for i in modelpred.keys():
            rst[i] = pd.DataFrame()
            for tkr in modelpred[i]['ticker'].unique(): 
                subdf = modelpred[i] [modelpred[i]['ticker'] == tkr ]

                rst[i].loc[tkr, 'correct'] = len(subdf[subdf[label] == subdf['y_pred']]) / len(subdf)
                rst[i].loc[tkr, 'wrong'] = len(subdf[(subdf[label] != subdf['y_pred']) & (subdf['y_pred']!=0)])/ len(subdf)
                rst[i].loc[tkr, 'miss'] =  (len(subdf[(subdf[label] != subdf['y_pred']) & (subdf['y_pred']==0)])/ len(subdf))
                rst[i].loc[tkr, 'predicted_n'] = abs(subdf['y_pred']).sum()
                rst[i] = rst[i][rst[i]['predicted_n']!=0]
                rst[i].sort_values(by ='correct',ascending = False, inplace = True)
        return rst

# Get final result

In [9]:
fe = feature_engineering()

In [10]:
fe.df = fe.calculate_y_label( 15, 0.075, True)


In [11]:
fe.df =  fe.calculate_col_diff ('OPEN_INT_TOTAL_CALL', 'OPEN_INT_TOTAL_PUT', 'OPEN_INT_DIFF')
fe.df =  fe.calculate_col_diff ('PX_BID', 'PX_ASK', 'BID_ASK_DIFF')
fe.df = fe.calculate_xd_ma( ['PX_LAST','VOLUME'], [5,90,260],True)


In [12]:
fe.df = fe.calculate_xd_sharpe( ['PX_LAST'], [5,90,260],True)
fe.df = fe.calculate_structural_break ( ['VOLUME','PX_LAST','OPEN_INT_DIFF'], True)
fe.df = fe.calculate_group_diff_agg(['PX_LAST','VOLUME'], 'industry',True)

MemoryError: Unable to allocate 10.5 GiB for an array with shape (18068, 78306) and data type float64

In [ ]:
fe.df = fe.calculate_bollinger_band( ['PX_LAST'], True)    
fe.df = fe.calculate_col_diff ('FUND_FLOW_90_dma', 'FUND_FLOW_5_dma', 'FUND_FLOW_90_5_DIFF')
fe.df = fe.calculate_col_diff ('PX_LAST_260_dma', 'PX_LAST_90_dma', 'PX_LAST_260_90_DIFF')

In [ ]:
fe.df = fe.df.resample('W').last()

In [ ]:
varlist = ['BID_ASK_DIFF','PX_LAST_5_dsharpe','PX_LAST_90_dsharpe','PX_LAST_260_dsharpe','PX_LAST_sb','PX_LAST_bollinger_upper',
          'PE_RATIO','OPEN_INT_DIFF']
y = 'y_15_d'

In [ ]:
fe.df = fe.generate_final_df(varlist + [y])
fe.df = fe.convert_y_label([y])

# Get final prediction

In [ ]:
finalmodels = pd.read_pickle( 'finalmodels.pkl')
rst = pd.read_pickle('rst.pkl')

for i in rst.keys(): 
    rst[i]['predicted_correct_n'] = rst[i]['correct'] * rst[i]['predicted_n']

In [ ]:
mb = model_building( fe.df)
latest_rst  = mb.get_latest_prediction(finalmodels['gb'], fe.df[input_features+ ['ticker']],  rst['gb'][['correct','predicted_n']] )
past_rst =  mb.get_past_prediction(finalmodels['gb'], fe.df[input_features+ ['ticker']],  rst['gb'][['correct','predicted_n']] )

In [ ]:
bt = backtest()
backtest =bt.portfolio_stats_matrix(fe.daily_rts, past_rst,'long only','weighted')
backtest[-1].plot(title = 'Backtest sharpe ' + i + ':' + str(backtest[2]))

In [ ]:
latest_rst.head(30)